# Таск 1 - вопросно ответная система

In [1]:
from langchain_openai import ChatOpenAI


# Таск 2 - Поиск ссылок в интернете

In [73]:
import json
import subprocess
import base64
from typing import Optional, Dict, Any
from pydantic import BaseModel

class YandexSearchConfig(BaseModel):
    api_key: str
    folder_id: str = "b1giur1tl3ajqq25bacd"
    search_type: str = "SEARCH_TYPE_RU"
    family_mode: str = "FAMILY_MODE_MODERATE"
    fix_typo_mode: str = "FIX_TYPO_MODE_ON"
    sort_mode: str = "SORT_MODE_BY_RELEVANCE"
    sort_order: str = "SORT_ORDER_DESC"
    group_mode: str = "GROUP_MODE_DEEP"
    groups_on_page: int = 3
    docs_in_group: int = 1
    max_passages: int = 3
    region: Optional[str] = None
    l10n: str = "LOCALIZATION_RU"
    response_format: str = "FORMAT_XML"
    grpc_host: str = "searchapi.api.cloud.yandex.net:443"

class YandexSearchAPI:
    def __init__(self, config: YandexSearchConfig):
        self.config = config

    def search(self, query: str, page: int = 0) -> Optional[str]:
        """Формирует gRPC-запрос, выполняет его и возвращает результат как строку."""
        
        body = json.dumps({
            "query": {
                "search_type": self.config.search_type,
                "query_text": query,
                "family_mode": self.config.family_mode,
                "page": page,
                "fix_typo_mode": self.config.fix_typo_mode
            },
            "sort_spec": {
                "sort_mode": self.config.sort_mode,
                "sort_order": self.config.sort_order
            },
            "group_spec": {
                "group_mode": self.config.group_mode,
                "groups_on_page": self.config.groups_on_page,
                "docs_in_group": self.config.docs_in_group
            },
            "max_passages": self.config.max_passages,
            "region": self.config.region,
            "l10n": self.config.l10n,
            "folder_id": self.config.folder_id,
            "response_format": self.config.response_format,
            "user_agent": "Python gRPC Client"
        })

        grpc_command = [
            "grpcurl",
            "-rpc-header", f"Authorization: Api-Key {self.config.api_key}",
            "-d", body,
            self.config.grpc_host,
            "yandex.cloud.searchapi.v2.WebSearchService/Search"
        ]

        try:
            result = subprocess.run(grpc_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            if result.returncode != 0:
                print(f"Ошибка gRPC-вызова: {result.stderr}")
                return None
            
            response_data = json.loads(result.stdout)

            if "rawData" in response_data:
                decoded_data = base64.b64decode(response_data["rawData"]).decode("utf-8")
                return decoded_data
            else:
                print("Ошибка: rawData отсутствует в ответе")
                return None

        except Exception as e:
            print(f"Ошибка выполнения gRPC-запроса: {e}")
            return None


In [76]:

search_api = YandexSearchAPI(config)
query_text = "В каком году Университет ИТМО был включён в число Национальных исследовательских университетов России?\n1. 2007\n2. 2009\n3. 2011\n4. 2015"
response = search_api.search(query_text)
print(response)  # Выведет XML-ответ


<?xml version="1.0" encoding="utf-8"?>
<yandexsearch version="1.0">
<request>
<query>В каком году Университет ИТМО был включён в число Национальных исследовательских университетов России?
1. 2007
2. 2009
3. 2011
4. 2015</query>
<page>0</page>
<sortby order="descending" priority="no">rlv</sortby>
<maxpassages>3</maxpassages>
<groupings>
<groupby attr="d" mode="deep" groups-on-page="3" docs-in-group="1" curcateg="-1"/>
</groupings>
</request>
<response date="20250130T212218">
<reqid>1738272137920836-13950889689152355344-balancer-l7leveler-kubr-yp-vla-130-BAL</reqid>
<found priority="phrase">4382364</found>
<found priority="strict">4382364</found>
<found priority="all">4382364</found>
<found-human>Нашлось 4 млн ответов</found-human>
<is-local>no</is-local>
<results>
<grouping attr="d" mode="deep" groups-on-page="3" docs-in-group="1" curcateg="-1">
<page first="1" last="4">0</page>
<group>
<categ attr="d" name="wikipedia.org"/>
<doccount>64</doccount>
<relevance priority="all"/>
<doc touch

In [78]:
import xml.etree.ElementTree as ET
root = ET.fromstring(response)

documents = []

for doc in root.findall(".//doc"):

    title_elem = doc.find("title")
    title = "".join(title_elem.itertext()) if title_elem is not None else ""

    url = doc.find("url").text if doc.find("url") is not None else ""
    
    passages = []
    for passages_elem in doc.findall(".//passages"):
        for passage_elem in passages_elem.findall(".//passage"):
            passages.append("".join(passage_elem.itertext()))
    
    extended_text_elem = doc.find(".//extended-text")
    extended_text = "".join(extended_text_elem.itertext()) if extended_text_elem is not None else ""
    
    documents.append({
        "title": title,
        "url": url,
        "passages": passages,
        "extended_text": extended_text
    })

for doc in documents:
    print(f"Заголовок: {doc['title']}")
    print(f"Ссылка: {doc['url']}")
    print(f"Фрагменты: {' | '.join(doc['passages'])}")
    print(f"Расширенный: {doc['extended_text']}")
    print("-" * 80)

Заголовок: Университет ИТМО — Википедия
Ссылка: https://ru.wikipedia.org/wiki/%D0%A3%D0%BD%D0%B8%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%82%D0%B5%D1%82_%D0%98%D0%A2%D0%9C%D0%9E
Фрагменты: Статус национального исследовательского университета (НИУ) ИТМО присвоили в 2009 году. | В 2012 году Университет ИТМО и американский RSV Venture Partners создали акселератор IT-инноваций и фонд, бюджет которого подразумевал инвестиции до $6 млн[126]'[127].
Расширенный текст: Университет ИТМО (в советское время — ЛИТМО, Ленинградский институт точной механики и оптики) — российское федеральное государственное автономное учебное заведение высшего и послевузовского образования, с 2009 года — национальный исследовательский университет (НИУ). Основан в 1900 году в Санкт-Петербурге.
--------------------------------------------------------------------------------
Заголовок: ИСТОРИЯ УНИВЕРСИТЕТА ИТМО
Ссылка: https://itmo.ru/ru/page/211/istoriya_universiteta_itmo.htm
Фрагменты: «Национальный исследовательский универси

In [54]:
from lxml import etree


# Разбираем XML
root = etree.fromstring(response.encode())

# Находим все <group>
documents = []
for group in root.xpath(".//group"):
    doc = group.find("doc")
    if doc is not None:
        # Убираем теги <hlword> внутри заголовка
        title_element = doc.find("title")
        title = "".join(title_element.itertext()).strip() if title_element is not None else "Нет заголовка"

        url = doc.findtext("url", "Нет ссылки")

        # Извлекаем фрагменты текста из <passage>
        passages = [p.text.strip() for p in doc.findall(".//passage") if p.text]

        documents.append({
            "title": title,
            "url": url,
            "passages": passages
        })

# Выводим результаты
for doc in documents:
    print(f"Заголовок: {doc['title']}")
    print(f"Ссылка: {doc['url']}")
    print(f"Фрагменты: {' | '.join(doc['passages']) if doc['passages'] else 'Нет фрагментов'}")
    print("-" * 80)


Заголовок: Университет ИТМО — Википедия
Ссылка: https://ru.wikipedia.org/wiki/%D0%A3%D0%BD%D0%B8%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%82%D0%B5%D1%82_%D0%98%D0%A2%D0%9C%D0%9E
Фрагменты: Статус | В 2012
--------------------------------------------------------------------------------
Заголовок: ИСТОРИЯ УНИВЕРСИТЕТА ИТМО
Ссылка: https://itmo.ru/ru/page/211/istoriya_universiteta_itmo.htm
Фрагменты: « | В
--------------------------------------------------------------------------------
Заголовок: Университет ИТМО - Википедия
Ссылка: https://tr-page.yandex.ru/translate?lang=en-ru&url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FITMO_University
Фрагменты: В
--------------------------------------------------------------------------------


In [81]:
import json
import subprocess
import base64
import xml.etree.ElementTree as ET
from typing import Optional, List, Dict, Any
from pydantic import BaseModel

class YandexSearchConfig(BaseModel):
    api_key: str
    folder_id: str = "b1giur1tl3ajqq25bacd"
    search_type: str = "SEARCH_TYPE_RU"
    family_mode: str = "FAMILY_MODE_MODERATE"
    fix_typo_mode: str = "FIX_TYPO_MODE_ON"
    sort_mode: str = "SORT_MODE_BY_RELEVANCE"
    sort_order: str = "SORT_ORDER_DESC"
    group_mode: str = "GROUP_MODE_DEEP"
    groups_on_page: int = 3
    docs_in_group: int = 1
    max_passages: int = 3
    region: Optional[str] = None
    l10n: str = "LOCALIZATION_RU"
    response_format: str = "FORMAT_XML"
    grpc_host: str = "searchapi.api.cloud.yandex.net:443"

class YandexSearchAPI:
    def __init__(self, config: YandexSearchConfig):
        self.config = config

    def search(self, query: str, page: int = 0) -> Optional[str]:
        """Формирует gRPC-запрос, выполняет его и возвращает результат как строку."""
        body = json.dumps({
            "query": {
                "search_type": self.config.search_type,
                "query_text": query,
                "family_mode": self.config.family_mode,
                "page": page,
                "fix_typo_mode": self.config.fix_typo_mode
            },
            "sort_spec": {
                "sort_mode": self.config.sort_mode,
                "sort_order": self.config.sort_order
            },
            "group_spec": {
                "group_mode": self.config.group_mode,
                "groups_on_page": self.config.groups_on_page,
                "docs_in_group": self.config.docs_in_group
            },
            "max_passages": self.config.max_passages,
            "region": self.config.region,
            "l10n": self.config.l10n,
            "folder_id": self.config.folder_id,
            "response_format": self.config.response_format,
            "user_agent": "Python gRPC Client"
        })

        grpc_command = [
            "grpcurl",
            "-rpc-header", f"Authorization: Api-Key {self.config.api_key}",
            "-d", body,
            self.config.grpc_host,
            "yandex.cloud.searchapi.v2.WebSearchService/Search"
        ]

        try:
            result = subprocess.run(grpc_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            if result.returncode != 0:
                print(f"Ошибка gRPC-вызова: {result.stderr}")
                return None
            
            response_data = json.loads(result.stdout)
            
            if "rawData" in response_data:
                decoded_data = base64.b64decode(response_data["rawData"]).decode("utf-8")
                return decoded_data
            else:
                print("Ошибка: rawData отсутствует в ответе")
                return None
        
        except Exception as e:
            print(f"Ошибка выполнения gRPC-запроса: {e}")
            return None

class YandexSearchParser:
    @staticmethod
    def parse(xml_response: str) -> List[Dict[str, Any]]:
        """Парсит XML-ответ и извлекает документы."""
        root = ET.fromstring(xml_response)
        documents = []

        for doc in root.findall(".//doc"):
            title_elem = doc.find("title")
            title = "".join(title_elem.itertext()) if title_elem is not None else ""

            url = doc.find("url").text if doc.find("url") is not None else ""
            
            passages = []
            for passages_elem in doc.findall(".//passages"):
                for passage_elem in passages_elem.findall(".//passage"):
                    passages.append("".join(passage_elem.itertext()))
            
            extended_text_elem = doc.find(".//extended-text")
            extended_text = "".join(extended_text_elem.itertext()) if extended_text_elem is not None else ""
            
            documents.append({
                "title": title,
                "url": url,
                "passages": passages,
                "extended_text": extended_text
            })

        return documents


In [84]:
config = YandexSearchConfig(api_key="AQVN2pO8Ve4QyhSutF_Db5QjDcbhdBOb-owXkrQK")

search_api = YandexSearchAPI(config)
query = "Университет ИТМО национальный исследовательский университет"
response = search_api.search(query)

if response:
    documents = YandexSearchParser.parse(response)
    for doc in documents:
        print(doc)

{'title': 'Университет ИТМО', 'url': 'https://itmo.ru/', 'passages': ['ИТМО совместно с «Газпром нефтью» разрабатывают новые материалы, сенсоры, роботов и другие технические и цифровые решения для нефтегазовой отрасли.'], 'extended_text': ''}
{'title': 'Университет ИТМО — Википедия', 'url': 'https://ru.wikipedia.org/wiki/%D0%A3%D0%BD%D0%B8%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%82%D0%B5%D1%82_%D0%98%D0%A2%D0%9C%D0%9E', 'passages': [], 'extended_text': ''}
{'title': 'Национальный исследовательский университет ИТМО, ВУЗ...', 'url': 'https://yandex.ru/maps/org/natsionalny_issledovatelskiy_universitet_itmo/1051701714/', 'passages': ['Национальный исследовательский университет ИТМО, факультет безопасности информационных технологий (Песочная наб., 14, Санкт-Петербург), вуз в Санкт‑Петербурге.'], 'extended_text': ''}


# Таск 3 - Получение новостей из RSS лент

In [79]:
import requests
from bs4 import BeautifulSoup
url = "https://news.itmo.ru/ru/"
headers = {"User-Agent": "Mozilla/5.0"}
    
    # Загружаем HTML
response = requests.get(url, headers=headers)
if response.status_code != 200:
    print(f"Ошибка: {response.status_code}")


soup = BeautifulSoup(response.text, "html.parser")
page_count = soup.find_all("a", class_="NewsCard__Card-sc-hw6n7n-0 iWYCxV card")


In [80]:
page_count

[]

# Таск 4 - дополнение RAG

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

def load_embeddings(type):
    """
    Загружает модель эмбеддингов HuggingFace для заданного устройства (CPU или GPU).

    :param type: Тип устройства для загрузки модели ('cpu' или 'cuda').
    :return: Экземпляр HuggingFaceEmbeddings.
    """
    model_id = 'intfloat/multilingual-e5-large'
    if type == 'cpu':
        model_kwargs = {'device': 'cpu'}
    else:
        model_kwargs = {'device': 'cuda'}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_id,
        model_kwargs=model_kwargs
    )
    return embeddings

def load_database(_embedding, faiss_idx):
    """
    Загружает или создает локальную базу данных FAISS для хранения векторных представлений.

    :param _embedding: Экземпляр модели эмбеддингов.
    :param faiss_idx: Путь к локальному индексу FAISS.
    :return: Экземпляр FAISS базы данных.
    """
    if os.path.exists(faiss_idx) and os.path.isdir(faiss_idx):
        db = FAISS.load_local(faiss_idx, _embedding, allow_dangerous_deserialization=True)
    else:
        db = FAISS.from_texts("Hello pipl", _embedding)
        db.save_local(faiss_idx)
    return db


In [ ]:
embending = load_embeddings('cpu')
database = load_database(embending, CONFIG.FAISS_INDEX_PATH)

# Таск 5 - обернуть в граф

In [87]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage

class AgentState(TypedDict):
    messages: Sequence[BaseMessage]
    category: str
    api_call_count: int

In [88]:
def call_main_classifer(state: AgentState):
    """
    Узел классификации:
    Использует LLM для классификации последнего запроса пользователя на категории: weather, finance или general.
    """
    messages = state["messages"]
    user_msg = messages[-1].content

    prompt = SystemMessage(content=(
            '''Классифицируй обращения пользователя в подходящую категорию.
            Категории:
            {'presentation' : 'Пользователь просит создать презентацию',
            'leasson_create' : 'Пользователь просит дополнить пункты плана обучения учебным материалом',
            'general' : 'Остальное'}
            В ответе укажи только категорию, ключ.'''
        ))
    query = HumanMessage(content=user_msg)

    response = llm.invoke([prompt, query])
    category = response.content.strip().lower()
    if category not in ["presentation", "leasson_create"]:
        category = "general"

    return {"messages": state["messages"], "category": category, "api_call_count": state["api_call_count"]}

In [85]:


workflow = StateGraph(AgentState)

# Добавляем узлы
workflow.add_node("rag_req", call_main_classifer)
#
workflow.add_node("leasson_create_agent", call_leasson_create_agent)
#
workflow.add_node("presentation_agent", call_presentation_create_agent)
#
workflow.add_node("general_agent", call_general_gpt_agent)

# Начинаем с классификатора
workflow.set_entry_point("main_classifier")

# Добавляем ветвление в зависимости от категории
workflow.add_conditional_edges(
    "main_classifier",
    route_by_category,
    {
        "leasson_create_agent": "leasson_create_agent",
        "presentation_agent": "presentation_agent",
        "general_agent": "general_agent"
    }
)

workflow.add_edge("leasson_create_agent", END)
workflow.add_edge("presentation_agent", END)
workflow.add_edge("general_agent", END)

app = workflow.compile()

display(Image(app.get_graph().draw_mermaid_png()))
